Iterators, Generators and Classic Coroutines

In [10]:
import sentence
from sentence import Sentence
s = Sentence('"The time has come," the Walrus said,')

In [15]:
for words in s:
    print(words,sep='\t')

list(s)
s[0]
s[5]
s[-1]

The
time
has
come
the
Walrus
said


'said'

Why Sequences Are Iterable: The iter Function

In [1]:
#Whenever Python needs to iterate over an object x, it automatically calls iter(x).

#The iter built in function:

#1 Checks whether the object implements __iter__ and calls that to obtain an iterator.

#2 If __iter__ is not implemented but __getitem__ is then iter() creates an iterator that tries to fetch items by index starting form 0

#3 If that fails, Python raised TypeError usually sayign C object is not iterable where c is the class of the target object.

Imp

In [2]:
#Hence clearly an object is not only considered iterable when it necessarily inherits the iter method but also iterable if it only inherits the getitem method.

In [3]:
#Hence all the sequnce objects have either the iter or the getitem method

In [ ]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()


spam_can  =Spam()
iter(spam_can)
list(spam_can)

#checking if iterable

from collections import abc
isinstance(spam_can, abc.Iterable)
class GooseSpam:
    def __iter__(self):
        pass
from collections import abc
issubclass(GooseSpam, abc.Iterable)
goose_spam_can = GooseSpam()
isinstance(goose_spam_can, abc.Iterable)

#Imp

In [21]:
#the most accurate way to check whether an object x is iterable is to call iter(x) and handle a TypeError exception if it isnt. THis is more accurate than using isinstance and issubclass because iter(x) aso considers the legacy __getitem__ method while Iterable ABC does not.


In [9]:
#Using iter with a Callable
from random import randint
def d6():
    return randint(1,6)  #Generates random integer between 1 and 6

d6_iter = iter(d6,1)   #Iterates until 1 is reached

for roll in d6_iter:
    print(roll)

6
5


In [ ]:
#One useful application of the second form of iter() is to build a block reader. For example reading fixed width blocks form a binary database file until the end of file is readched:

from functools import partial

with open('mydata.db', 'rb') as f:
    read64 = partial(f.read, 64)
    for block in iter(read64, b''):
        process_block(block)

Iterables Versus Iterators

In [19]:
#iterable

#Any object form which the iter built in funciton can obtain an iterator. Objects implementing __iter__ method returning an iterator are iterable. Sequences are always iterable as are objects implementing a __getitem__ method that accepts 0 based indexes.

#Here is a simple for loop iterating over a str. The str 'ABC' is the iterable here. You dont see it but there is an iterator behind the curtain.
from collections import abc
s = 'ABC'
for char in s:
    print(char)


A
B
C


In [33]:
a = iter(range(3))
a.__next__()
s = 'ABC'
iter(s).__next__()
s.__nex

'A'

In [54]:
#If there was no for statement and we had to emulate the for machinery by hand with a while loop this is what wed have to write


s = 'ABC'
a = []

while len(a)==0:
    b = iter(s)
    c = iter(s)
    while len(a)<len(s):
        a.append(c.__next__())
        try:
            print(b.__next__())
        except StopIteration:
            break
        
#The below is given in the book
#>>> s = 'ABC'
#>>> it = iter(s)  
#>>> #while True:
#...     try:
#...         print(next(it))  
#...     #except StopIteration:  
#...       #  del it  
#...         #break#

#Build an iterator it form the iterable

#Repeatedly call nex on the iterator ot o;btain the nex iterm.



A
B
C


In [1]:
#StopIteartion signals that the iterator is exhausted. This exception is handles internally byt the iter() built in that is part of th elogic of for loops an other iteration contexts like list compreshension, iteratble unpacking etc.

#Pythons standard itnerface for an iterator has two methods:

#__next__
#Return the next item in the serise, raising StopIteration if there are no more.

#__iter__
#returns self this allows iterators to be used where an iterable is expected for example in a for loop

#That interface is formalized in teh collections.abc,.Iterator ABC., Which declares the __next__ abstract method and subclasses Iterable where the abstract __iter__ method is declared



In [2]:
#__subclasshook__ supports structural type checks with isinstance and issubclass. We saw it in "Structural Typing with ABCs"

#_check_methods traverses the __mro__ of the class to check whether the methods are inmplemented in its base classes. Its defined in that smae module of the abc. If methods are implemented the C class will be recognized as a virtual subclass of Iterator. In other words 


##IMP -- Because the only methods required of an iterator are __next__ and __iter__ there is no way to check whether there are remaining iterms, other tha to call next() and catch StopIteration. Also its not possible to reset an iterator. If you need to start over you need to call iter() on the iterable that built the iterator in the first place. 

#Calling iter() on the iterator itself wont helpo either because as mentioned Iterator.__iter__ is implemented by returning self, so thsi will not reset a depleted iterator.

#That minimal interaface is sensible because in reality not all iterators are resetable . Fo rexample if an iterator is reading packets form the network theres no way to rewind it

Sentence Classes with `__iter__`